# Telegram Module — Test Notebook

Test the Telegram pipeline components: reply generation, message building, and sending.

Sections 1-3 run without API keys. Section 4 requires a Telegram bot token.

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "tg" else Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

## 1. Generate a Reply (single message)

The `run()` function takes a text string and returns a reply.

In [ ]:
from tg.handlers.build_replies import run, generate_reply

# Single message reply (default: echoes back)
message = "Hello, bot! How are you?"
reply = run(message)
print(f"Input:  {message}")
print(f"Reply:  {reply}")

In [ ]:
# Test with different messages
test_messages = [
    "What time is it?",
    "Tell me a joke",
    "Help me with my order #12345",
]

for msg in test_messages:
    reply = run(msg)
    print(f"  In:  {msg}")
    print(f"  Out: {reply}")
    print()

## 2. Build Replies from Trigger Entries

The `build_replies()` function processes a batch of trigger entries.

In [ ]:
from tg.handlers.build_replies import build_replies
import json

# Example trigger entries (simulating messages from Telegram)
trigger_entries = [
    {
        "id": "entry-001",
        "chat": {"id": 123456789, "type": "private", "first_name": "Alice"},
        "user": {"id": 111, "first_name": "Alice", "username": "alice123"},
        "message": {"message_id": 1, "text": "What are your hours?"},
    },
    {
        "id": "entry-002",
        "chat": {"id": 987654321, "type": "private", "first_name": "Bob"},
        "user": {"id": 222, "first_name": "Bob", "username": "bob456"},
        "message": {"message_id": 2, "text": "Do you deliver?"},
    },
    {
        "id": "entry-003",
        "chat": {"id": 123456789, "type": "private", "first_name": "Alice"},
        "user": {"id": 111, "first_name": "Alice", "username": "alice123"},
        "message": {"message_id": 3, "text": "Thanks!"},
    },
]

replies = build_replies(trigger_entries)
print(f"Built {len(replies)} replies:\n")
print(json.dumps(replies, indent=2, default=str))

In [ ]:
# Filter by chat ID — only process Alice's messages
alice_replies = build_replies(trigger_entries, filter_chat_id=123456789)
print(f"Alice's replies: {len(alice_replies)}")
for r in alice_replies:
    print(f"  -> {r['reply']['text']}")

## 3. Queue Manager

Test the JSON queue operations (load, save, append, clear).

In [ ]:
from tg.utils.queue_manager import load_queue, save_queue, append_queue, clear_queue
import tempfile, os

test_queue_path = os.path.join(tempfile.gettempdir(), "test_queue.json")

# Start fresh
clear_queue(test_queue_path)
print(f"Empty queue: {load_queue(test_queue_path)}")

# Append entries
append_queue(test_queue_path, {"id": 1, "text": "first message"})
append_queue(test_queue_path, {"id": 2, "text": "second message"})

entries = load_queue(test_queue_path)
print(f"After appending 2 entries: {entries}")

# Save a full replacement
save_queue(test_queue_path, [{"id": 99, "text": "replaced"}])
print(f"After save: {load_queue(test_queue_path)}")

# Clear
clear_queue(test_queue_path)
print(f"After clear: {load_queue(test_queue_path)}")

## 4. Send Replies via Telegram (requires bot token)

This sends real messages via the Telegram API.
Set your bot token and chat ID below before running.

In [ ]:
import asyncio
from tg.actions.send_replies import send_all

# Replace with your actual values
BOT_TOKEN = "YOUR_BOT_TOKEN_HERE"
CHAT_ID = 123456789  # Your chat ID

reply_entries = [
    {
        "chat": {"id": CHAT_ID},
        "reply": {"text": "Hello from the test notebook!"},
    },
]

if BOT_TOKEN != "YOUR_BOT_TOKEN_HERE":
    sent = asyncio.run(send_all(BOT_TOKEN, reply_entries))
    print(f"Sent {sent} message(s)")
else:
    print("Set BOT_TOKEN and CHAT_ID above to test sending.")